In [1]:
import cx_Oracle
import pandas as pd

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from PC.L_VW_TRAIN_CON_LAGS" 
df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_full = pd.concat(df, ignore_index=True)
conn.close()
print(df_full.shape)
# Imprimir las primeras filas del DataFrame completo
print(df_full.head())
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)
# Imprimir el número de filas y columnas del DataFrame completo
print(f"Número de filas: {df_full.shape[0]}, Número de columnas: {df_full.shape[1]}")

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_238248\2129643656.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


(10143267, 17)
   PERIODO  CUSTOMER_ID  PRODUCT_ID         TN  TN_MAS_DOS  TN_LAG_01  \
0   201701        10001       20001   99.43861    92.46537        NaN   
1   201702        10001       20001  198.84365    13.29728   99.43861   
2   201703        10001       20001   92.46537   101.00563  198.84365   
3   201704        10001       20001   13.29728   128.04792   92.46537   
4   201705        10001       20001  101.00563   101.20711   13.29728   

   TN_LAG_02  TN_LAG_03  TN_LAG_04  TN_LAG_05  TN_LAG_06  TN_LAG_07  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2   99.43861        NaN        NaN        NaN        NaN        NaN   
3  198.84365   99.43861        NaN        NaN        NaN        NaN   
4   92.46537  198.84365   99.43861        NaN        NaN        NaN   

   TN_LAG_08  TN_LAG_09  TN_LAG_10  TN_LAG_11  TN_LAG_12  
0        NaN        NaN        NaN        NaN        NaN  
1

In [2]:
# Usando este código como base para un modelo de regresión con LightGBM
# agregar hojas de LightGBM

import lightgbm as lgb

# Variables predictoras y objetivo
X = df_full[['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID', 'TN', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 
             'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 'TN_LAG_12']]
y = df_full['TN_MAS_DOS']

# Crear el dataset de LightGBM
train_data = lgb.Dataset(X, label=y)

# Definir parámetros para regresión
params = { 
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1 }

# Entrenar el modelo
model = lgb.train(params, train_data, num_boost_round=100)

print("Modelo de regresión entrenado con todo el dataset.")

Modelo de regresión entrenado con todo el dataset.


In [4]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "SELECT * FROM l_vw_PRED_CON_LAGS" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_238248\1133623266.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [5]:
# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID', 'TN', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 
             'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 'TN_LAG_12']]
predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES'] = predictions
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")
# Guardar el modelo entrenado
model.save_model('modelo_regresion.txt')


   PERIODO  PRODUCT_ID  CUSTOMER_ID         TN  TN_LAG_01  TN_LAG_02  \
0   201912       20001        10001  180.21938  236.65556  176.02980   
1   201912       20002        10001  334.03714  547.87849  430.90803   
2   201912       20003        10001  137.98717  135.69192  196.18531   
3   201912       20004        10001   12.94020   27.58851   37.88891   
4   201912       20005        10001    7.66693   11.01719    7.98907   

   TN_LAG_03  TN_LAG_04  TN_LAG_05  TN_LAG_06  TN_LAG_07  TN_LAG_08  \
0  109.05244   33.63991  144.78714   65.92436  439.90647  364.37071   
1  213.36148  148.91108  103.12062  151.12081  264.55349  155.81927   
2  101.61982  121.06458  105.81480   78.79703   74.71874   86.54509   
3   20.57492    8.33349   16.04585   34.26047   27.99741   17.84712   
4    5.66966    2.51269    5.41195    3.20851    4.25654    1.72238   

   TN_LAG_09  TN_LAG_10  TN_LAG_11  TN_LAG_12  PREDICCIONES  
0  130.54927  309.90610  386.60688  254.62373    207.263859  
1  220.19153  27

In [6]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_DATOS_PREDICCION con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
# Hacer Update L_DATOS_PREDICCION set TN_PREDICCION = NULL
update_query = """
    UPDATE L_DATOS_PREDICCION
    SET TN_PREDICCION = NULL
"""
cursor.execute(update_query)
# Hacer commit para aplicar el cambio de NULL
conn.commit()
# Imprimir mensaje de inicio de actualización
print("Iniciando actualización de la tabla L_DATOS_PREDICCION con las nuevas predicciones.")

# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_DATOS_PREDICCION
        SET TN_PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_DATOS_PREDICCION con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla L_DATOS_PREDICCION completada con las nuevas predicciones.")


Iniciando actualización de la tabla L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 0 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 10000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 20000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 30000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 40000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 50000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 60000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 70000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 80000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 90000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 100000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 110000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actual